In [2]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import matplotlib
import sklearn as sk
matplotlib.style.use('ggplot')

# From Excel training sheets
training_values = pd.read_csv('/Users/noahmitchellward/Desktop/EECS_351/Project/Training_Set_Values.csv')
training_labels = pd.read_csv('/Users/noahmitchellward/Desktop/EECS_351/Project/Training_Set_Labels.csv')

# Get the full training set
train = pd.merge(training_labels, training_values, how='inner', on='id')


In [3]:
# Replacing zero/null construction years with the average of construction year
constrMean = train[train['construction_year'] > 0]['construction_year'].mean()

train2 = train

train2.loc[train2['construction_year'] <= 0, 'construction_year'] = constrMean
train2.loc[train2['construction_year'] == 0]

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group


In [25]:
# messing around
pd.crosstab(train['district_code'],train['status_group'])

status_group,functional,functional needs repair,non functional
district_code,,,
0,4,2,17
1,6558,1287,4358
2,6203,844,4126
3,4954,699,4345
4,5595,509,2895
5,2479,194,1683
6,2055,232,1787
7,2014,214,1115
8,589,53,401


In [32]:
# Removing columns that are deemed not useful, incoherent, or have little predictive power

# We can see here that extraction_type is the most specific instance of the extraction method, with
# extraction_type_group is the most general, and extraction_type_class in the middle. We use the most specific case.
print(pd.crosstab(train['extraction_type'],train['extraction_type_group']))

print(pd.crosstab(train['extraction_type'],train['extraction_type_class']))

del train2['extraction_type_group']
del train2['extraction_type_class']

# Next is quantity and quantity_group - they contain the same data, so we delete quantity_group
print(pd.crosstab(train['quantity'],train['quantity_group']))

del train2['quantity_group']


extraction_type_group      afridev  gravity  india mark ii  india mark iii  \
extraction_type                                                              
afridev                       1770        0              0               0   
cemo                             0        0              0               0   
climax                           0        0              0               0   
gravity                          0    26780              0               0   
india mark ii                    0        0           2400               0   
india mark iii                   0        0              0              98   
ksb                              0        0              0               0   
mono                             0        0              0               0   
nira/tanira                      0        0              0               0   
other                            0        0              0               0   
other - mkulima/shinyanga        0        0              0      

KeyError: 'extraction_type_group'

In [33]:
# recorded_by is all the same value; it contains no information
print(train2['recorded_by'].value_counts())

del train2['recorded by']



GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64


In [34]:
# Function to delete columns from dataframe
def deleteColumns(df, catList):
    for category in catList:
        df = df.drop(category,1)
    return df

In [36]:
deleteColumns(train2,['num_private','payment']).columns

Index(['id', 'status_group', 'amount_tsh', 'date_recorded', 'funder',
       'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'management', 'management_group', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [37]:
train2.columns

Index(['id', 'status_group', 'amount_tsh', 'date_recorded', 'funder',
       'gps_height', 'installer', 'longitude', 'latitude', 'wpt_name',
       'num_private', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'recorded_by', 'scheme_management', 'scheme_name', 'permit',
       'construction_year', 'extraction_type', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [9]:
svmCols = pd.concat([train2['longitude'], train2['latitude']], axis=1)
svmCols = pd.concat([svmCols, train2['gps_height']], axis=1)
svmCols

,longitude,latitude,gps_height
0,34.938093,-9.856322e+00,1390
1,34.698766,-2.147466e+00,1399
2,37.460664,-3.821329e+00,686
3,38.486161,-1.115530e+01,263
4,31.130847,-1.825359e+00,0
5,39.172796,-4.765587e+00,0
6,33.362410,-3.766365e+00,0
7,32.620617,-4.226198e+00,0
8,32.711100,-5.146712e+00,0
9,30.626991,-1.257051e+00,0


In [24]:
from sklearn.svm import NuSVC

clf = NuSVC()
clf.fit(svmCols, training_labels)


ValueError: bad input shape (59400, 2)

In [18]:
clf.predict([[2., 2.]])

array([1])